# Fine-tuning API 활용 예시

원본 파일: `llm-modeling-lab.jsonl`

```json
{
  "input": "아인슈페너 레귤러 사이즈로 부탁드리고, 콩국수 한 그릇 주세요.",
  "output": "- 분석 결과 0: 음식명:아인슈페너,옵션:레귤러,수량:1\n- 분석 결과 1: 음식명:콩국수,수량:한 그릇"
}
```

In [5]:
import jsonlines

# 파일 준비

In [1]:
system_message = "너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다."

In [6]:
# 파일 경로 설정
input_filepath = '../local_data/llm-modeling-lab.jsonl'  # 원본 JSONL 파일 경로
output_filepath = '../local_data/order-understanding-data.jsonl'  # 변환된 JSONL 파일 저장 경로

# 시스템 메시지
system_message = "너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다."

# 변환을 위한 함수 정의
def convert_to_finetuning_format(input_filepath, output_filepath, system_message):
    # 입력 파일 읽기 및 출력 파일 쓰기
    with jsonlines.open(input_filepath, 'r') as infile, jsonlines.open(output_filepath, 'w') as outfile:
        for data in infile:
            # input과 output을 Fine-tuning API 포맷에 맞게 변환
            formatted_data = {
                "messages": [
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": data['input']},
                    {"role": "assistant", "content": data['output']}
                ]
            }
            # 변환된 데이터를 출력 파일에 작성
            outfile.write(formatted_data)

# 변환 함수 실행
convert_to_finetuning_format(input_filepath, output_filepath, system_message)

print(f"데이터가 '{output_filepath}' 파일에 성공적으로 저장되었습니다.")


데이터가 '../local_data/order-understanding-data.jsonl' 파일에 성공적으로 저장되었습니다.


데이터 분할
- 앞쪽 2,000개: train 
- 뒷쪽 1,000개: validation

**TODO**
- 200개 / 100개 조합으로 갯수를 줄여서 해보자

In [7]:
!head -n 2000 ../local_data/order-understanding-data.jsonl > ../local_data/order-understanding-train.jsonl
!tail -n 1000 ../local_data/order-understanding-data.jsonl > ../local_data/order-understanding-validation.jsonl

# Client 설정

In [9]:
from openai import OpenAI
import openai
import os
with open("../specs/.openai_api_key", "r") as f:
    ss = f.readline().strip()

os.environ["OPENAI_API_KEY"] = ss
client = OpenAI()

# 파일 업로드

In [10]:
training_file_name = "../local_data/order-understanding-train.jsonl"
validation_file_name = "../local_data/order-understanding-validation.jsonl"

In [12]:
def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id


training_file_id = upload_file(training_file_name, "fine-tune")
validation_file_id = upload_file(validation_file_name, "fine-tune")

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-XjPP5QD7iIl540TMZQPmjEQo
Validation file ID: file-62bejvcHwJT3t2e2W4w41nny


# 모델 미세 튜닝 job 생성

In [13]:
MODEL = "gpt-4o-mini-2024-07-18"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=MODEL,
    suffix="order-understanding",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-hkVw9Qi7hsCRD0GzdpMMCywU
Status: validating_files


# 진행 상태 확인

In [15]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-hkVw9Qi7hsCRD0GzdpMMCywU
Status: running
Trained Tokens: None


In [21]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 1485/1500: training loss=0.02
Step 1486/1500: training loss=0.10
Step 1487/1500: training loss=0.05
Step 1488/1500: training loss=0.08
Step 1489/1500: training loss=0.08
Step 1490/1500: training loss=0.03
Step 1491/1500: training loss=0.03
Step 1492/1500: training loss=0.06
Step 1493/1500: training loss=0.12
Step 1494/1500: training loss=0.07
Step 1495/1500: training loss=0.10
Step 1496/1500: training loss=0.06
Step 1497/1500: training loss=0.03
Step 1498/1500: training loss=0.02
Step 1499/1500: training loss=0.05
Step 1500/1500: training loss=0.06, validation loss=0.03, full validation loss=0.09
Checkpoint created at step 500
Checkpoint created at step 1000
New fine-tuned model created
The job has successfully completed


In [22]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError(
        "Fine-tuned model ID not found. Your job has likely not been completed yet."
    )

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-4o-mini-2024-07-18:sungshin-women-s-university:order-understanding:AChwOffJ


# 결과 모델 활용

In [23]:
completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[{"role":"system", "content":system_message},{"role": "user", "content": "아이스 초코라떼 두 잔 주세요. 그리고 함께 먹을 수 있는 빵 하나도 주세요."}]
)
print(completion.choices[0].message)


ChatCompletionMessage(content='주문 문장:     - 분석 결과 0: 음식명:아이스 초코라떼,수량:두 잔 \n주문 문장:     - 분석 결과 1: 음식명:빵,수량:하나 ', refusal=None, role='assistant', function_call=None, tool_calls=None)
